<a href="https://colab.research.google.com/github/cagBRT/timeSeries/blob/main/8c_time_series_for_DNNs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Functions for preparing time series data for DNNs

In [ ]:
!git clone -l -s https://github.com/cagBRT/timeSeries.git cloned-repo
%cd cloned-repo

In [ ]:
import pandas as pd
import numpy as np
from numpy import array
import matplotlib.pyplot as plt
import pickle

from sklearn.preprocessing import MinMaxScaler
import statsmodels.tsa.stattools as sts
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import tensorflow as tf
from keras.src.layers.serialization import preprocessing_normalization
from keras.models import Model,Sequential
from keras.layers import Dense,LSTM,Dropout

In [ ]:
dataset=pd.read_csv("temperature.csv")
dataset.head()

In [ ]:
dataset.tail()

In [ ]:
df2 = dataset[["month", "day", "year"]].copy()
df_date=pd.to_datetime(df2)
df_date=pd.DataFrame(df_date)

In [ ]:
dfAT=dataset["AverageTemperatureFahr"]
df_date.insert(1,"AverageTemperatureFahr",dfAT )


In [ ]:
df_date

In [ ]:
df_date_renamed=df_date.rename(columns={0:"date"})
df_date_renamed

In [ ]:
df_nomissing=df_date_renamed.dropna()
df_nomissing

In [ ]:
df_nomissing=df_nomissing.set_index('date')
df_nomissing.columns
df_nomissing



---



---



In [ ]:
def to_supervised(train,n_input,n_out):
    #flaten data
    data=train
    X,y=list(),list()
    in_start=0
    for _ in range(len(data)):
        in_end=in_start+ n_input
        out_end=in_end + n_out
        if out_end<=len(data):
            x_input=data[ in_start:in_end,0]
            x_input=x_input.reshape((len(x_input)))
            X.append(x_input)
            y.append(data[in_end:out_end,0])
        in_start+=1
    return array(X), array(y)

To use to_supervise: <br>
Convert the data to a numpy array

In [ ]:
data_NM=np.array(df_nomissing)
n_step=1
lags = 7
X,y=to_supervised(data_NM, lags,n_step)

In [ ]:
X[0]

In [ ]:
y[0]

In [ ]:
X[1]



---



---



For this function, the series is used as a Pandas dataframe

In [ ]:
def series_to_supervised(data,n_in,n_out):

    df = pd.DataFrame(data)
    cols = list()
    for i in range(n_in,0,-1): cols.append(df.shift(i))
    for i in range(0, n_out): cols.append(df.shift(-i))
    agg = pd.concat(cols,axis=1)
    agg.dropna(inplace=True)

    return agg.values

In [ ]:
data_ready=series_to_supervised(df_nomissing,lags,n_step)

In [ ]:
data_ready[0]

In [ ]:
X= np.delete(data_ready,7,1)
X[0]

In [ ]:
X[1]

In [ ]:
y=data_ready[0][7]
y